In [14]:
"""
penalty1: float, >0，当无责判有责，用于控制司机上诉的代价
penalty2: float，>0, 当有责判无责，用于控制乘客上诉的代价
guiltyRate: float, 0~1, 订单中实际的有责率

"""

population = 1000
normalTime = 6 
guilty_when_true = 1
innocent_when_true = 0

def TimeMatrix(guiltyRate=0.1, penalty_rider=2, penalty_driver=1):
    import pandas as pd
    import numpy as np
    import warnings
    warnings.filterwarnings('ignore')
    
    if guiltyRate>1 or guiltyRate<0:
        print('Reset Guilty Rate')
        return 
      
    topN = np.linspace(0,1,11)
    recall = np.linspace(0,1,11)[::-1]
    pplGuilty = population * guiltyRate
    pplInnocent = population - pplGuilty
    guilty_riderComplain = normalTime * penalty_rider
    innocent_driverComplain = guilty_when_true + normalTime*penalty_driver
    
    ncols = len(topN)+1
    nrows = len(recall)
    matrix = [[0 for i in range(ncols)] for j in range(nrows)] 
    for i in range(ncols-1):
        matrix[0][i+1] = recall[i]
    for i in range(nrows):
        matrix[i][0] = topN[i]
    for i in range(1,nrows):
        for j in range(1,ncols):
            if guiltyRate*matrix[0][j]<=matrix[i][0]:
                matrix[i][j] = pplGuilty*matrix[0][j]*guilty_when_true + \
                pplGuilty*(1-matrix[0][j])*guilty_riderComplain + \
                (population*matrix[i][0] - pplGuilty*matrix[0][j])*innocent_driverComplain
            else:
                matrix[i][j] = 'N/A'
            
    data = pd.DataFrame(matrix[1:nrows],columns=range(ncols))
    data.columns = ['TopN']+[str(int(i*100)) + '%' for i in recall]
    data.ix[:,'TopN'] = [str(int(i*100))+'%' for i in topN[1:]]
    data.iloc[-1,2:] = 'N/A'
    data.set_index('TopN', inplace=True)
    return data

stats = TimeMatrix(0.3)
stats

,100%,90%,80%,70%,60%,50%,40%,30%,20%,10%,0%
TopN,,,,,,,,,,,
10%,N/A,N/A,N/A,N/A,N/A,N/A,N/A,2680,3220,3760,4300
20%,N/A,N/A,N/A,N/A,1760,2300,2840,3380,3920,4460,5000
30%,300,840,1380,1920,2460,3000,3540,4080,4620,5160,5700
40%,1000,1540,2080,2620,3160,3700,4240,4780,5320,5860,6400
50%,1700,2240,2780,3320,3860,4400,4940,5480,6020,6560,7100
60%,2400,2940,3480,4020,4560,5100,5640,6180,6720,7260,7800
70%,3100,3640,4180,4720,5260,5800,6340,6880,7420,7960,8500
80%,3800,4340,4880,5420,5960,6500,7040,7580,8120,8660,9200
90%,4500,5040,5580,6120,6660,7200,7740,8280,8820,9360,9900


In [34]:
'''
topn:float, (0,1]
recalln:float, [0,1]
penalty1: float, >0，当无责判有责，用于控制司机上诉的代价
penalty2: float，>0, 当有责判无责，用于控制乘客上诉的代价
guiltyRate: float, [0,1], 订单中实际的有责率

'''
population = 1000
normalTime = 6 
guilty_when_true = 1
innocent_when_true = 0

def assessment(topN,recalln,guiltyRate=0.1,penalty_rider=2, penalty_driver=1):
    import pandas as pd
    import numpy as np
    
    if topN == 1 and recalln != 1:
        return 'N/A'
    if 0<topN<=1 and 0<=recalln<=1 and 0<=guiltyRate<=1:
        pplGuilty = population*guiltyRate
        pplInnocent = population - pplGuilty
        guilty_rider_complain = normalTime * penalty_rider
        innocent_driver_complain = guilty_when_true + normalTime * penalty_driver
        time = pplGuilty * recalln * guilty_when_true + \
                pplGuilty*(1-recalln)*guilty_rider_complain + \
                (population*topN - pplGuilty*recalln)*innocent_driver_complain
        time_saved = (population * normalTime - time)/(population * normalTime)
        if guiltyRate*recalln>topN:
            return 'N/A'
        print('will increase efficiency by {}%'.format(time_saved*100))
    else:
        print('reset parameter')

stats = assessment(0.75,0.3)
stats
    

will increase efficiency by 1.5%
